In [4]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('data/chinook.db')

customers = pd.read_sql_query("SELECT * FROM customers", conn)
invoices = pd.read_sql_query("SELECT * FROM invoices", conn)

merged_df = pd.merge(customers, invoices, on='CustomerId', how='inner')

invoice_counts = merged_df.groupby(['CustomerId', 'FirstName', 'LastName']).size().reset_index(name='InvoiceCount')

print(invoice_counts)

conn.close()


    CustomerId  FirstName      LastName  InvoiceCount
0            1       Luís     Gonçalves             7
1            2     Leonie        Köhler             7
2            3   François      Tremblay             7
3            4      Bjørn        Hansen             7
4            5  František   Wichterlová             7
5            6     Helena          Holý             7
6            7     Astrid        Gruber             7
7            8       Daan       Peeters             7
8            9       Kara       Nielsen             7
9           10    Eduardo       Martins             7
10          11  Alexandre         Rocha             7
11          12    Roberto       Almeida             7
12          13   Fernanda         Ramos             7
13          14       Mark       Philips             7
14          15   Jennifer      Peterson             7
15          16      Frank        Harris             7
16          17       Jack         Smith             7
17          18   Michelle   

In [6]:
movie_df = pd.read_csv('data/movie.csv')

df_color = movie_df[['director_name', 'color']]

df_reviews = movie_df[['director_name', 'num_critic_for_reviews']]

left_join_df = pd.merge(df_color, df_reviews, on='director_name', how='left')
print("Left Join Row Count:", len(left_join_df))

outer_join_df = pd.merge(df_color, df_reviews, on='director_name', how='outer')
print("Full Outer Join Row Count:", len(outer_join_df))

Left Join Row Count: 30300
Full Outer Join Row Count: 30300


In [12]:
titanic_df = pd.read_excel('data/titanic.xlsx') 

grouped_titanic = titanic_df.groupby('Pclass').agg({
    'Age': 'mean',          
    'Fare': 'sum',           
    'PassengerId': 'count'   
}).rename(columns={
    'Age': 'AverageAge',
    'Fare': 'TotalFare',
    'PassengerId': 'PassengerCount'
}).reset_index()

print(grouped_titanic)

   Pclass  AverageAge   TotalFare  PassengerCount
0       1   38.233441  18177.4125             216
1       2   29.877630   3801.8417             184
2       3   25.140620   6714.6951             491


In [11]:
movie_df = pd.read_csv('data/movie.csv')

grouped_movies = movie_df.groupby(['color', 'director_name']).agg({
    'num_critic_for_reviews': 'sum',  
    'duration': 'mean'            
}).rename(columns={
    'num_critic_for_reviews': 'TotalCriticReviews',
    'duration': 'AverageDuration'
}).reset_index()

grouped_movies


,color,director_name,TotalCriticReviews,AverageDuration
0,Black and White,Akira Kurosawa,153.0,202.0
1,Black and White,Aleksey German,121.0,177.0
2,Black and White,Alex Garland,489.0,108.0
3,Black and White,Alexander Payne,433.0,115.0
4,Black and White,Alfred Hitchcock,434.0,119.0
...,...,...,...,...
2485,Color,Zoran Lisinac,17.0,108.0
2486,Color,Álex de la Iglesia,71.0,104.0
2487,Color,Émile Gaudreault,67.0,92.0
2488,Color,Éric Tessier,9.0,99.0


In [16]:
flights_data = {
    'Year': [2020, 2020, 2021, 2021, 2021],
    'Month': [1, 2, 1, 2, 3],
    'FlightNum': [101, 102, 103, 104, 105],
    'ArrDelay': [5.0, -3.0, 10.0, None, 0.0],
    'DepDelay': [0.0, 7.0, None, 15.0, 3.0]
}

flights_df = pd.DataFrame(flights_data)

flights_df.to_parquet('data/flights.parquet', index=False)

grouped_flights = flights_df.groupby(['Year', 'Month']).agg({
    'FlightNum': 'count',         
    'ArrDelay': 'mean',          
    'DepDelay': 'max'           
}).rename(columns={
    'FlightNum': 'TotalFlights',
    'ArrDelay': 'AvgArrivalDelay',
    'DepDelay': 'MaxDepartureDelay'
}).reset_index()

print(grouped_flights)

   Year  Month  TotalFlights  AvgArrivalDelay  MaxDepartureDelay
0  2020      1             1              5.0                0.0
1  2020      2             1             -3.0                7.0
2  2021      1             1             10.0                NaN
3  2021      2             1              NaN               15.0
4  2021      3             1              0.0                3.0


In [18]:
titanic_df = pd.read_excel('data/titanic.xlsx')

def classify_age(age):
    if pd.isna(age):
        return 'Unknown'
    return 'Child' if age < 18 else 'Adult'

titanic_df['Age_Group'] = titanic_df['Age'].apply(classify_age)

print(titanic_df[['Age', 'Age_Group']].head())

    Age Age_Group
0  22.0     Adult
1  38.0     Adult
2  26.0     Adult
3  35.0     Adult
4  35.0     Adult


In [22]:
employee_df = pd.read_csv('data/employee.csv')

employee_df['Normalized_Salary'] = employee_df.groupby('DEPARTMENT')['BASE_SALARY'].transform(
    lambda x: (x - x.min()) / (x.max() - x.min())
)

employee_df.head()

,UNIQUE_ID,POSITION_TITLE,DEPARTMENT,BASE_SALARY,RACE,EMPLOYMENT_TYPE,GENDER,EMPLOYMENT_STATUS,HIRE_DATE,JOB_DATE,Normalized_Salary
0,0,ASSISTANT DIRECTOR (EX LVL),Municipal Courts Department,121862.0,Hispanic/Latino,Full Time,Female,Active,2006-06-12,2012-10-13,1.000000
1,1,LIBRARY ASSISTANT,Library,26125.0,Hispanic/Latino,Full Time,Female,Active,2000-07-19,2010-09-18,0.000000
2,2,POLICE OFFICER,Houston Police Department-HPD,45279.0,White,Full Time,Male,Active,2015-02-03,2015-02-03,0.116351
3,3,ENGINEER/OPERATOR,Houston Fire Department (HFD),63166.0,White,Full Time,Male,Active,1982-02-08,1991-05-25,0.192491
4,4,ELECTRICIAN,General Services Department,56347.0,White,Full Time,Male,Active,1989-06-19,1994-10-22,0.479189


In [24]:
movie_df = pd.read_csv('data/movie.csv')

def classify_duration(duration):
    if pd.isna(duration):
        return 'Unknown'
    elif duration < 60:
        return 'Short'
    elif 60 <= duration <= 120:
        return 'Medium'
    else:
        return 'Long'

movie_df['Length_Category'] = movie_df['duration'].apply(classify_duration)

print(movie_df[['duration', 'Length_Category']].head())

   duration Length_Category
0     178.0            Long
1     169.0            Long
2     148.0            Long
3     164.0            Long
4       NaN         Unknown


In [25]:
def filter_survivors(df):
    return df[df['Survived'] == 1]

def fill_age(df):
    df['Age'] = df['Age'].fillna(df['Age'].mean())
    return df

def add_fare_per_age(df):
    df['Fare_Per_Age'] = df['Fare'] / df['Age']
    return df

titanic_pipeline = (
    titanic_df
    .pipe(filter_survivors)
    .pipe(fill_age)
    .pipe(add_fare_per_age)
)

print(titanic_pipeline[['Survived', 'Age', 'Fare', 'Fare_Per_Age']].head())

   Survived   Age     Fare  Fare_Per_Age
1         1  38.0  71.2833      1.875876
2         1  26.0   7.9250      0.304808
3         1  35.0  53.1000      1.517143
8         1  27.0  11.1333      0.412344
9         1  14.0  30.0708      2.147914


C:\Users\posit\AppData\Local\Temp\ipykernel_17984\1624185624.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age'] = df['Age'].fillna(df['Age'].mean())
C:\Users\posit\AppData\Local\Temp\ipykernel_17984\1624185624.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Fare_Per_Age'] = df['Fare'] / df['Age']


In [28]:
flights_df = pd.read_parquet('data/flights.parquet')

def filter_long_delays(df):
    return df[df['DepDelay'] > 30]

def calc_delay_per_hour(df):
    df = df[df['AirTime'] > 0]
    df['Delay_Per_Hour'] = df['DepDelay'] / (df['AirTime'] / 60)
    return df

flights_df['AirTime'] = [120, 150, 180, 200, 240] 

flights_pipeline = (
    flights_df
    .pipe(filter_long_delays)
    .pipe(calc_delay_per_hour)
)

print(flights_pipeline[['DepDelay', 'AirTime', 'Delay_Per_Hour']].head())

Empty DataFrame
Columns: [DepDelay, AirTime, Delay_Per_Hour]
Index: []
